In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

2.5.0
2.5.0


In [2]:
from nonconformist.nc import NcFactory
from nonconformist.cp import IcpClassifier

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 15

# Florida voter
df = pd.read_csv('../dataverse_files/fl_reg_name_race.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df[df.race.isin(['multi_racial', 'native_indian', 'other', 'unknown']) == False].sample(SAMPLE, random_state=21)
del df

# Additional features
sdf['name_last'] = sdf.name_last.str.title()

sdf

,name_last,name_first,race
841323,Torres,Jose,hispanic
1408926,Da Silva,Amanda,nh_white
1733118,Mc Ghee,Sandra,nh_white
13104513,Karam,MELINDA,nh_white
9156114,Brewer,LAIA,nh_black
...,...,...,...
3076722,Antunez Avila,Robert,hispanic
10023679,Davis,WYATT,nh_white
5846252,Scott,Jacquelyn,nh_white
5959131,Parton,Douglas,nh_white


##  Preprocessing the input data

In [4]:
# last name only
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)
X_train, X_calib, y_train, y_calib = train_test_split(X_train, y_train, test_size=.4, random_state=10, stratify=y_train)

num_words = 1166
Max feature len = 26, Avg. feature len = 5


## Setting up the data

In [5]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_calib), 'calibration sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_calib = sequence.pad_sequences(X_calib, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_calib shape:', X_calib.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('y_train shape:', y_train.shape)
print('y_calib shape:', y_calib.shape)
print('y_test shape:', y_test.shape)

480000 train sequences
320000 calibration sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (480000, 20)
X_calib shape: (320000, 20)
X_test shape: (200000, 20)
4 classes
y_train shape: (480000,)
y_calib shape: (320000,)
y_test shape: (200000,)


In [6]:
from nonconformist.base import ClassifierAdapter
from nonconformist.nc import ClassifierNc

class MyClassifierAdapter(ClassifierAdapter):
    def __init__(self, model, fit_params=None):
        super(MyClassifierAdapter, self).__init__(model, fit_params)

        
    def fit(self, x, y):
        '''
            x is a numpy.array of shape (n_train, n_features)
            y is a numpy.array of shape (n_train)
            
            Here, do what is necessary to train the underlying model
            using the supplied training data
        '''        
        model = Sequential()
        model.add(Embedding(num_words, 32, input_length=feature_len))
        model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(num_classes, activation='softmax'))

        # using sparse_categorical_crossentropy to meet requirements for y
        #    as stated in the documentation of the class
        model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
        model.fit(x, y, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
        
        model.save('../models/fl_voter_reg/lstm/fl_all_name_lstm_nonconform.h5')
    
    def predict(self, x):
        '''
            Obtain predictions from the underlying model
            
            Make sure this function returns an output that is compatible with
            the nonconformity function used. For default nonconformity functions,
            output from this function should be class probability estimates in
            a numpy.array of shape (n_test, n_classes)
        '''
        keras_model = keras.models.load_model('../models/fl_voter_reg/lstm/fl_all_name_lstm_nonconform.h5')
        results = keras_model.predict(x)
        return results
    
my_classifier = None # Initialize an object of your classifier's type
model = MyClassifierAdapter(my_classifier)
nc = ClassifierNc(model)
icp = IcpClassifier(nc)

In [7]:
# Training and calibrating the data using noncorformist
icp.fit(X_train, y_train)
icp.calibrate(X_calib, y_calib)

2021-11-21 19:33:37.984558: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
13500/13500 [==============================] - 278s 19ms/step - loss: 0.6896 - accuracy: 0.7496 - val_loss: 0.6127 - val_accuracy: 0.7759
Epoch 2/15
13500/13500 [==============================] - 247s 18ms/step - loss: 0.6150 - accuracy: 0.7744 - val_loss: 0.5944 - val_accuracy: 0.7837
Epoch 3/15
13500/13500 [==============================] - 270s 20ms/step - loss: 0.5977 - accuracy: 0.7807 - val_loss: 0.5835 - val_accuracy: 0.7889
Epoch 4/15
13500/13500 [==============================] - 254s 19ms/step - loss: 0.5833 - accuracy: 0.7858 - val_loss: 0.5734 - val_accuracy: 0.7892
Epoch 5/15
13500/13500 [==============================] - 247s 18ms/step - loss: 0.5729 - accuracy: 0.7890 - val_loss: 0.5690 - val_accuracy: 0.7914
Epoch 6/15
13500/13500 [==============================] - 247s 18ms/step - loss: 0.5676 - accuracy: 0.7906 - val_loss: 0.5651 - val_accuracy: 0.7944
Epoch 7/15
13500/13500 [==============================] - 247s 18ms/step - loss: 0.5626 - accuracy: 0.7920

In [8]:
# obtaining the predictions based on our model
results = icp.predict(X_test, significance=0.1)

In [9]:
# sample output
results[:10]

array([[False, False,  True,  True],
       [False, False,  True,  True],
       [False,  True, False,  True],
       [False, False, False,  True],
       [False, False, False,  True],
       [False,  True,  True,  True],
       [False, False, False,  True],
       [False, False, False,  True],
       [ True,  True,  True,  True],
       [False, False, False,  True]])

In [10]:
results.astype(int)

array([[0, 0, 1, 1],
       [0, 0, 1, 1],
       [0, 1, 0, 1],
       ...,
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

In [11]:
cnt_races = np.unique(results,axis=0)
cnt_races.shape

(15, 4)

## Sample prediction output

In [12]:
races = np.array(sorted(sdf.race.unique().tolist()))
races.shape

(4,)

In [13]:
test_sample = sdf.sample(frac=0.01, random_state=10)
test_sample.reset_index(inplace=True)
test_sample.drop(['index','name_first','name_last_name_first'], axis=1, inplace=True)
test_sample

,name_last,race
0,Liberto,nh_white
1,Weill,nh_white
2,Smith,nh_white
3,Jones,nh_white
4,Trinidad,hispanic
...,...,...
9995,Longo,nh_white
9996,Reynoso Luna,hispanic
9997,Yearwood,nh_white
9998,Turner,nh_black


In [14]:
encod_arr = np.array(test_sample.name_last.apply(lambda c: find_ngrams(c, NGRAMS)))
encod_arr = sequence.pad_sequences(encod_arr, maxlen=feature_len)

out_arr = icp.predict(encod_arr, significance=0.1 )

In [15]:
pred_series = []
for i in range(out_arr.shape[0]):
    pred_series.append(races[[x for x,i in enumerate(out_arr[i]) if i]].tolist())

In [16]:
test_sample['pred_race'] = pred_series

In [17]:
test_sample

,name_last,race,pred_race
0,Liberto,nh_white,[nh_white]
1,Weill,nh_white,[nh_white]
2,Smith,nh_white,"[nh_black, nh_white]"
3,Jones,nh_white,"[nh_black, nh_white]"
4,Trinidad,hispanic,[hispanic]
...,...,...,...
9995,Longo,nh_white,[nh_white]
9996,Reynoso Luna,hispanic,[hispanic]
9997,Yearwood,nh_white,"[nh_black, nh_white]"
9998,Turner,nh_black,"[nh_black, nh_white]"
